In [1]:
from simulation_loop import SimulationParams, simulate_many, Buffer, simulate_system
from scipy.stats import norm, uniform, gamma, expon
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
np.random.seed(42)

In [2]:
p_ = 0.55
t_ = np.linspace(0, 24, 1000)

class ArrivalDistribution:
    first_round_time = 9
    second_round_time = 15
    def __init__(self, p=0.5, offset=0):
        self.p = p
        self.first_peak = norm(loc=10 + offset, scale=1)
        self.second_peak = norm(loc=15 + offset, scale=1.5)

    def pdf(self, x):
        return self.p * self.first_peak.pdf(x) + (1 - self.p) * self.second_peak.pdf(x)

    def rvs(self, size=()):
        s1 = self.first_peak.rvs(size=size)
        s2 = self.second_peak.rvs(size=size)
        choice = uniform.rvs(size=size) < self.p
        return np.where(choice, s1, s2)

discharge_dist = ArrivalDistribution()
admission_dist = ArrivalDistribution(offset=1)

arrivals = admission_dist.rvs(size=1000)
discharges = discharge_dist.rvs(size=1000)

def service_time_dist():
    service_min_time = 5/60
    return service_min_time + expon(scale=1/60).rvs()


def transport_time_dist(distance_meters):
    if distance_meters == 0:
        return 0
    speed_meters_per_second = 1.4
    transport_time_seconds = gamma(a=distance_meters, scale=1/speed_meters_per_second).rvs()
    return transport_time_seconds / 3600

distances = np.array([
    [10, 50, 175, 150],
    [50, 10, 0, 0],
    [175, 0, 10, 0],
    [150, 0, 0, 10],
])

n_elevators = 3
discharge_dist = lambda: ArrivalDistribution().rvs(size=200)

# Sensitivity Analysis - Base case

## Queue lengths

In [ ]:
params = SimulationParams(
    discharge_dist,
    service_time_dist,
    transport_time_dist,
    distances,
    arrival_weights=np.array([0.8, 0.1, 0.1]),
    discharge_weights=np.array([0.2, 0.4, 0.4]),
    n_elevators=n_elevators,
    n_robots=1,
)

buffers, demands, arrival_times, discharge_times, df = simulate_many(params, n_iters=10)

threshold = 10  

metrics = ["Clean buffer (daily max)", "Dirty buffer (daily max)", "Bed demand (daily max)"]
percentiles = ["50th", "90th", "95th", "99th"]

print(f"Service level: Percentile ≤ {threshold} beds")
for m in metrics:
    print(f"\n{m}:")
    for p in percentiles:
        v = df.loc[m, p]
        status = "OK" if v <= threshold else "Exceeds"
        print(f"  {p:>4} percentile → {v:5.2f} → {status}")

# Extract per‑run maxima
clean_max = buffers[:, Buffer.CLEAN].max(axis=(1,2))
dirty_max = buffers[:, Buffer.DIRTY].max(axis=(1,2))
demand_max = demands.max(axis=(1,2))

# Plot histograms side by side
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
axs[0].hist(clean_max, bins=20, edgecolor="black")
axs[0].set(title="Clean Buffer (Daily Max)", xlabel="Beds", ylabel="Frequency")
axs[1].hist(dirty_max, bins=20, edgecolor="black")
axs[1].set(title="Dirty Buffer (Daily Max)", xlabel="Beds")
axs[2].hist(demand_max, bins=20, edgecolor="black")
axs[2].set(title="Bed Demand (Daily Max)", xlabel="Beds")
plt.tight_layout()
plt.show()

  0%|          | 0/10 [00:00<?, ?it/s]

 70%|███████   | 7/10 [00:05<00:02,  1.25it/s]


KeyboardInterrupt: 

# Increasing the number of bots

In [19]:
robot_range = [1, 2, 3, 5, 10]
n_iters = 5

clean_stats = []
dirty_stats = []

for n_robots in robot_range:
    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=n_robots,
    )

    buffers, *_ = simulate_many(params, n_iters=n_iters)

    # Compute max buffer size per run
    clean_vals = buffers[:, Buffer.CLEAN, :, :].mean(axis=(1, 2))
    dirty_vals = buffers[:, Buffer.DIRTY, :, :].mean(axis=(1, 2))

    # Compute percentiles across simulation runs
    clean_percentiles = np.percentile(clean_vals, [90, 95, 99])
    dirty_percentiles = np.percentile(dirty_vals, [90, 95, 99])

    clean_stats.append(clean_percentiles)
    dirty_stats.append(dirty_percentiles)

quantile_labels = ['90th', '95th', '99th']
robot_labels = [str(r) for r in robot_range]

clean_df = pd.DataFrame(clean_stats, index=robot_labels, columns=quantile_labels)
dirty_df = pd.DataFrame(dirty_stats, index=robot_labels, columns=quantile_labels)

print("\n=== Clean Buffer Percentiles ===")
print(clean_df.round(2))

print("\n=== Dirty Buffer Percentiles ===")
print(dirty_df.round(2))

100%|██████████| 5/5 [00:04<00:00,  1.09it/s]


=== Clean Buffer Percentiles ===
    90th  95th  99th
1   0.10  0.10  0.11
2   0.09  0.09  0.09
3   0.08  0.08  0.08
5   0.07  0.07  0.07
10  0.09  0.09  0.09

=== Dirty Buffer Percentiles ===
     90th   95th   99th
1   11.04  11.15  11.24
2   10.66  10.66  10.66
3   10.82  10.83  10.85
5   11.06  11.17  11.25
10  10.96  11.03  11.08


# Changing $\lambda$

In [26]:
lambdas = [1, 1.2, 1.5, 2, 0.001]  # lambda_arrival values
scales = 1 / np.array(lambdas)
n_iters = 5  # number of simulation runs

clean_stats = []
dirty_stats = []

for lam in scales:
    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=1,
    )

    # Run simulations
    buffers, *_ = simulate_many(params, n_iters=n_iters, lambda_arrival=lam)

    # Take mean buffer occupancy per simulation run
    clean_vals = buffers[:, Buffer.CLEAN, :, :].mean(axis=(1, 2))
    dirty_vals = buffers[:, Buffer.DIRTY, :, :].mean(axis=(1, 2))

    # Compute percentiles across runs
    clean_percentiles = np.percentile(clean_vals, [90, 95, 99])
    dirty_percentiles = np.percentile(dirty_vals, [90, 95, 99])

    clean_stats.append(clean_percentiles)
    dirty_stats.append(dirty_percentiles)

# Labels
quantile_labels = ['90th', '95th', '99th']
lambda_labels = [str(lam) for lam in lambdas]

# Create and print DataFrames
clean_df = pd.DataFrame(clean_stats, index=lambda_labels, columns=quantile_labels)
dirty_df = pd.DataFrame(dirty_stats, index=lambda_labels, columns=quantile_labels)

print("\n=== Clean Buffer Percentiles by lambda_arrival ===")
print(clean_df.round(2))

print("\n=== Dirty Buffer Percentiles by lambda_arrival ===")
print(dirty_df.round(2))

100%|██████████| 5/5 [00:03<00:00,  1.25it/s]


=== Clean Buffer Percentiles by lambda_arrival ===
       90th  95th  99th
1      0.09  0.09  0.09
1.2    0.12  0.12  0.13
1.5    0.14  0.14  0.14
2      0.09  0.10  0.10
0.001  0.10  0.10  0.10

=== Dirty Buffer Percentiles by lambda_arrival ===
        90th   95th   99th
1      10.96  11.08  11.17
1.2    10.46  10.47  10.47
1.5    10.58  10.70  10.80
2      11.04  11.14  11.21
0.001  11.08  11.10  11.12
